In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

## Reflect Tables into SQLALchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from scipy.stats import ttest_ind
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.types import Date


In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")
conn = engine.connect()
session = Session(bind=engine)

In [5]:
# reflect an existing database into a new model
Base = declarative_base()


In [6]:
#create class for measurement table

class Measurement(Base):
    __tablename__ = "measurement"
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(Date)
    prcp = Column(Float)
    tobs = Column(Float)

In [7]:
# reflect the tables
# View all of the classes that automap found,#create class for station table
class Station(Base):
    __tablename__ = "station"
    
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation =  Column(Float)

In [8]:
# Create our session (link) from Python to the DB
conn = engine.connect()
session = Session(bind=engine)

## Bonus Challenge Assignment: Temperature Analysis II

In [9]:
# This function called `calc_temps` will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, maximum, and average temperatures for that range of dates
def calc_temps(start_date, end_date):
    """TMIN, TAVG, and TMAX for a list of dates.
    
    Args:
        start_date (string): A date string in the format %Y-%m-%d
        end_date (string): A date string in the format %Y-%m-%d
        
    Returns:
        TMIN, TAVE, and TMAX
    """
    
    results = session.query(func.min(Measurement.tobs).label("min"), func.avg(Measurement.tobs).label("avg"), func.max(Measurement.tobs).label("max")).\
        filter(Measurement.date >= start_date).filter(Measurement.date <= end_date).all()
    
    temp_list = []
    for ttemp in results:
        temp_list.append(ttemp.min)
        temp_list.append(ttemp.avg)
        temp_list.append(ttemp.max)

    return temp_list
# For example
# display temp data for dates provided

print(calc_temps('2012-02-28', '2012-03-05'))

[62.0, 69.57142857142857, 74.0]


In [10]:
# Use the function `calc_temps` to calculate the tmin, tavg, and tmax 
# for a year in the data set
trip_temps = calc_temps("2017-06-10", "2017-06-20")
trip_temps

[72.0, 76.77464788732394, 83.0]

In [11]:
# Plot the results from your previous query as a bar chart. 
# Use "Trip Avg Temp" as your Title
# Use the average temperature for bar height (y value)
# Use the peak-to-peak (tmax-tmin) value as the y error bar (yerr)
keys = ["Min", "Avg", "Max"]

dict_temps = {keys[i]: trip_temps[i] for i in range(len(keys))}
dict_temps

temp_df = pd.DataFrame.from_dict(dict_temps, orient="index", columns=["Temp"])
temp_df


,Temp
Min,72.000000
Avg,76.774648
Max,83.000000


### Daily Rainfall Average

In [12]:
# Calculate the total amount of rainfall per weather station for your trip dates using the previous year's 
# matching dates.
# Sort this in descending order by precipitation amount and list the station, name, latitude, longitude, and elevation
prcp_2016 = session.query(Measurement.station, func.round(func.sum(Measurement.prcp), 2))\
    .filter(Measurement.date > '2016-06-10')\
    .filter(Measurement.date <= '2016-06-20')\
    .order_by(Measurement.date).group_by(Measurement.station).all()

prcp_2016

[('USC00513117', 1.05),
 ('USC00514830', 1.22),
 ('USC00516128', 6.27),
 ('USC00519281', 2.5),
 ('USC00519397', 0.08),
 ('USC00519523', 0.56),
 ('USC00517948', 0.6)]

### Daily Temperature Normals

In [13]:
# Use this function to calculate the daily normals 
# (i.e. the averages for tmin, tmax, and tavg for all historic data matching a specific month and day)

def daily_normals(date):
    """Daily Normals.
    
    Args:
        date (str): A date string in the format '%m-%d'
        
    Returns:
        A list of tuples containing the daily normals, tmin, tavg, and tmax
    
    """
    
    sel = [func.min(Measurement.tobs), func.avg(Measurement.tobs), func.max(Measurement.tobs)]
    return session.query(*sel).filter(func.strftime("%m-%d", Measurement.date) == date).all()

# For example
daily_normals("01-01")

[(62.0, 69.15384615384616, 77.0)]

In [14]:
# calculate the daily normals for your trip
# push each tuple of calculations into a list called `normals`

# Set the start and end date of the trip
start_date = '2017-08-01'
end_date = '2017-08-07'

# Use the start and end date to create a range of dates
# Strip off the year and save a list of strings in the format %m-%d
# Use the `daily_normals` function to calculate the normals for each date string 
# and append the results to a list called `normals`.
trip_dates = ["06-10", "06-11", "06-12", "06-13", "06-14", "06-15", "06-16", "06-17", "06-18", "06-19", "06-20"]
trip_results = []

for i, date in enumerate(trip_dates):
    
    trip_temps = daily_normals(date)
    
    trip_results.append(trip_temps)
    
print(trip_results)

[[(68.0, 75.14285714285714, 81.0)], [(69.0, 76.01960784313725, 84.0)], [(66.0, 75.66666666666667, 83.0)], [(68.0, 75.66666666666667, 81.0)], [(69.0, 75.62711864406779, 81.0)], [(67.0, 75.40350877192982, 81.0)], [(69.0, 75.41379310344827, 81.0)], [(68.0, 74.59322033898304, 81.0)], [(69.0, 74.60344827586206, 80.0)], [(70.0, 75.44067796610169, 82.0)], [(68.0, 73.96491228070175, 85.0)]]


In [15]:
# Load the previous query results into a Pandas DataFrame and add the `trip_dates` range as the `date` index
trip_df = pd.DataFrame.from_dict(trip_temps_dict, orient="index", columns=["Min", "Avg", "Max"])

trip_df


NameError: name 'trip_temps_dict' is not defined

In [ ]:
# Plot the daily normals as an area plot with `stacked=False`
trip_df.plot.area(stacked=False, alpha=0.2)
plt.ylabel("Temperature")
plt.xlabel("Date")
plt.savefig("Images/area_plot.png")
plt.show()

## Close Session

In [ ]:
session.close()